# Initialize SKAI Colab Kernel
This notebook sets up a new custom Colab kernel VM to run the SKAI notebook.

To use:
1. Open this notebook in Colab.
2. <font color='red'>**Connect to the custom kernel you want to initialize.**
<font color='black'>
3. Change settings as appropriate.
4. Run all code cells.

In [ ]:
#@title Settings

#@markdown Please enter the parameters of **google cloud platform account** to use:
GCP_PROJECT_ID = "" #@param {type:"string"}
GCP_REGION = "us-central1" #@param {type:"string"}
CLOUD_SERVICE_ACCOUNT = "" #@param {type:"string"}

#@markdown ---
#@markdown Following parameters are used to set up the private key access control.

#@markdown **We recommend not changing these parameters.**
PRIVATE_KEY_PATH = "/root/service-account-private-key.json" #@param {type:"string"}

# Create Service Account Key File
Because the custom kernel VM is shared among multiple users, users should not authenticate with GCP using their own accounts. Otherwise, their private credentials will be written to the kernel's filesystem for all other users to see.

Instead, the colab should authenticate as a service account. In order to do that, a private key for the service account should be created and uploaded to the kernel. This should be done either via Cloud's web console following [these instructions](https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-create-console), or the user can run `gcloud` on their workstation to create the key file. The key file should end with ".json".

After creating the service account, assign the following roles by following the instructions [here](https://cloud.google.com/iam/docs/manage-access-service-accounts#multiple-roles-console)
- Vertex AI Custom Code Service Agent
- Vertex AI Service Agent
- Vertex AI User
- Cloud Build Service Account
- Cloud Build Service Agent
- Compute Engine Service Agent
- Container Registry Service Agent
- Cloud Dataflow Service Agent
- Editor
- Cloud Pub/Sub Service Agent
- Storage Admin

Regardless of how the key is created, it should be uploaded to the kernel.

**Note 1:** For security, a new private key should be created for each kernel, instead of reusing one key for multiple kernels. This way, we can revoke a kernel's permissions by deleting the key.

**Note 2:** By default, newly created private keys never expire. For increased security, we suggest setting the key to expire after you will no longer need it, for example in 6 months. You can always generate a new key after the current one expires. See [here](https://cloud.google.com/iam/docs/service-accounts#key-expiry) for details.

# Upload Service Account Key File

In [ ]:
#@markdown Upload serive account key file

from google.colab import files

print('Use Cloud web console to create a service account private key, or run the following command on your workstation:\n')
print(f'gcloud iam service-accounts keys create service-account-private-key.json --iam-account={CLOUD_SERVICE_ACCOUNT}\n')
print('Then upload the created file to to colab')

uploaded = files.upload()
if len(uploaded) == 1:
  KEY_FILE = list(uploaded.keys())[0]
  !mv {KEY_FILE} {PRIVATE_KEY_PATH}


# Set Cloud Project and Service Account

In [ ]:
#@markdown 1. Set project name and default location.
#@markdown 2. Activate the service account.
%shell gcloud config set project {GCP_PROJECT_ID}
%shell gcloud auth activate-service-account {CLOUD_SERVICE_ACCOUNT} --key-file={PRIVATE_KEY_PATH}
%shell gcloud config set compute/region {GCP_REGION}
%shell export GOOGLE_APPLICATION_CREDENTIALS=$PRIVATE_KEY_PATH
